# Logica en la BDD

## Contenidos

* Vistas
* Triggers
* STORE procedures

EN desarrollo de software le llamamos **business logi**, a la logica del negoicio o simplementologica de las regla, algoritmos, etc. definen el sitema y que estan dados por lo qu quiere lograr la aplicacion.

Los motores máscomplejos comoPOstgres o MySQLienen esas funcionalidades que permiten implmentar parte de esa logica en la base de datos.



## VIEWS

Las vistas son tablas virtuales que se crean a partir de una consulta SQL. Son útiles para simplificar la consulta de datos, ya que permiten ocultar la complejidad de las consultas.

Primero tengamos en cuenta tres tablas:

```sql
CREATE TABLE Directores(
    nombre VARCHAR(50) PRIMARY KEY,
    pais VARCHAR(50),
    retirado boolean
)

CREATE TABLE Peliculas(
    nombre VARCHAR(50),
    director VARCHAR(50),
    año INT,
    FOREIGN KEY (director) REFERENCES Directores(nombre),
    PRIMARY KEY (nombre, director)
)

CREATE TABLE Evaluaciones(
    pelicula VARCHAR(50),
    director VARCHAR(50),
    fuente varchar(50),
    eval float,
    FOREIGN KEY (pelicula, director) REFERENCES Peliculas(nombre, director),
    PRIMARY KEY (pelicula, director, fuente)
);
```
## Consulta frecuente

Socre promedio de lapelicula "The Prestige" de "Christopher Nolan"

```sql
SELECT AVG(eval) AS score
FROM Evaluaciones
WHERE nombre = 'The Prestige' AND director = 'Christopher Nolan';
```

Ahora veamos esto como una vista:

```sql
CREATE VIEW PelEval AS
    SELECT pelicula, director,
    AVG(eval) AS score
    FROM Evaluaciones
    GROUP BY pelicula, director;
```

![image](img/peleval.png)


Las vistas definen una tabla "virtual", en base a los datos presentes en una tabla o tablas ya existentes. Las vistas no almacenan datos, solo definen una consulta que se ejecuta cada vez que se consulta la vista.

Estas ayudan a simplificar la logica y el codigo de las consultas,por ejemplo tenemosdos equivalencias, una consulta anidada y una con vista:

```sql
SELECT promedio
FROM (
    SELECT pelicula, director,
    AVG(eval) AS promedio
    FROM Evaluaciones
    GROUP BY pelicula, director
)
WHERE pelicula = 'The Prestige' AND director = 'Christopher Nolan';
```
EQUIVALENTE A:
```sql
CREATE VIEW PelEval AS
    SELECT pelicula, director,
    AVG(eval) AS promedio
    FROM Evaluaciones
    GROUP BY pelicula, director

SELECT promedio
FROM PelEval
WHERE pelicula = 'The Prestige' AND director = 'Christopher Nolan';
```

## Eliminando vistas

```sql
DROP VIEW PelEval;
```

## Para que sirven las vistas?

Abstracción:
* Reducir la complejidad de consultas grandes.
* Evitar repetición de consultas frecuentes.

Mantenibilidad:
* Más fácil de manejar que la gestión de datos
duplicados o redundantes.
* Más fácil de optimizar y mantener que una
consulta repetida.
* Es más lento que tener las tablas de verdad!

## Vistas materializadas

Las vistas materializadas general la misma abstraccion que una tabla nomral pero con la ventaja de que son más rapidas.

```sql
CREATE MATERIALIZED VIEW PelEval AS
    SELECT pelicula, director,
    AVG(eval) AS promedio
    FROM Evaluaciones
    GROUP BY pelicula, director;
```

El problema que deemos actualizarlas cada que se actualiza la tabla original.

```sql
REFRESH MATERIALIZED VIEW PelEval;
```

como automatizamos? con triggers!


## Triggers

* Procedimientos en la base de datos que se
gatillan cada vez que se ejecuta algún evento.

* Contribuyen a forzar ciertas restricciones más
complejas y a mantener la consistencia de la BD.

SINTAXIS:

```sql
--El evento puede ser 
{BEFORE | AFTER | INSTEAD OF} {CREATE| DELETE | UPDATE}

-- Podemos ejecutar para cada fila o 1 vez por evento

{FOR EACH ROW | FOR EACH STATEMENT}

```

EJemplo: Queremos disminur el stock de un produco cada vez que se crea una venta.

```sql
CREATE TRIGGER disminuir_stock
AFTER CREATE ON Ventas
FOR EACH ROW
BEGIN
    UPDATE Productos
    SET stock = stock - 1
    WHERE id = NEW.producto_id;
END;
```

PAra la view que veiamos antes:

```sql
CREATE TRIGGER actualizar_promedio
AFTER CREATE ON Evaluaciones
FOR EACH STATEMENT
BEGIN
    REFRESH MATERIALIZED VIEW PelEval;
END;
```




## Lenguaje PROCEDURAL en SQL

SQL es un lenguaje declarativo, pero en algunos motores de bases de datos se puede usar un lenguaje procedural para definir funciones y procedimientos almacenados.

* RECORDAR: Lenguaje declarativo significa que le decimos a la base de datos qué queremos, no cómo hacerlo. Lenguaje procedural significa que le decimos a la base de datos cómo hacerlo. 

Basicamente en uno "escribimos" en el otro usamoslopps, if, etc, lo calsico.



## Stored Procedures

Son bloques de código que se almacenan en la base de datos y se pueden ejecutar desde cualquier cliente que se conecte a la base de datos.

Sirven paraejctar logiica compleja y repetitiva en la dbms

![image](img/stored.png)

## Stored Procedores o Funciones

Sintaixs:

```sql
CREATE or REPLACE Function <nombre_funcion> (<argumentos>) RETURNS
<tipo_retorno> AS
$$
DECLARE
<declaracion de variables>
BEGIN
<sentencias SQL>
END
$$ language plpgsql
```

Prcedimineto para insertar una fila a la tabla personas:

```sql
CREATE OR REPLACE FUNCTION insertar_persona (rut varchar, nombre varchar, apellido
varchar) RETURNS void AS
$$
BEGIN
INSERT INTO personas VALUES (rut,nombre,apellido);
END
$$ language plpgsql
```

Podemos iterar sobre resultados de consultas y usar eso para
procesar e insertar datos a otras tablas:

```sql
CREATE OR REPLACE FUNCTION transferencia_nombres() RETURNS void AS $$
DECLARE
tupla RECORD;
concat varchar;
BEGIN
FOR tupla IN SELECT * FROM Personas LOOP
concat = tupla.nombre || tupla.apellido;
insert into personascompleto values (tupla.rut, concat);
END LOOP;
END
$$ language plpgsql
```
### Consultas dinámicas
Podemos retornar ocnsultas completas y demas usar los argumentos de la funcion para generarlas de forma dinámica:

```sql
CREATE OR REPLACE FUNCTION vuelos_desde (c_origen varchar)
RETURNS TABLE (ciudad_destino varchar(50), horas integer) AS $$
BEGIN
RETURN QUERY EXECUTE ’
SELECT ciudad_destino, horas
FROM Vuelo
WHERE ciudad_origen = $1’
USING c_origen;
RETURN; END
$$ language plpgsql
```

## Difucultades de los stored procedures

Problemas al migrar la base de datos.